# Green Mail Agent Demonstration

This notebook demonstrates the **Green Mail Agent**, a scripted agent that simulates normal office email usage patterns. The agent periodically sends emails to other users, representing typical workplace communication.

## Overview

The Green Mail Agent is designed to create realistic email traffic patterns in PrimAITE simulations:

| **Component** | **Purpose** | **Key Features** |
|:-------------:|:-----------:|:----------------:|
| **Green Mail Agent** | Normal user simulation | Periodic email sending, realistic content |
| **Email Client** | Application interface | SMTP sending, POP3 receiving |
| **SMTP Server** | Email delivery | Message routing, mailbox management |
| **Network Infrastructure** | Communication backbone | Routing, DNS resolution |

## Learning Objectives

1. **Agent Configuration**: Understand how to configure green mail agents
2. **Email Patterns**: Observe realistic email communication patterns
3. **System Integration**: See how agents interact with email infrastructure
4. **Monitoring**: Use built-in display methods to observe agent behavior
5. **Scenario Design**: Learn to create multi-agent email scenarios

## Table of Contents

- [Environment Setup](#environment-setup)
- [Load Mail Scenario](#load-mail-scenario)
- [Agent Configuration Review](#agent-configuration-review)
- [Run Email Simulation](#run-email-simulation)
- [Monitor Agent Activity](#monitor-agent-activity)
- [Examine Email Traffic](#examine-email-traffic)
- [Agent Behavior Analysis](#agent-behavior-analysis)
- [Summary and Extensions](#summary-and-extensions)

## Environment Setup

First, we'll set up the PrimAITE environment and import the necessary components for our green mail agent demonstration.

In [ ]:
# Initialize PrimAITE environment
!primaite setup

In [ ]:
# Import required components
import yaml
from pathlib import Path
from prettytable import PrettyTable
import logging

# PrimAITE core components
import primaite
from primaite import PRIMAITE_CONFIG
from primaite.game.game import PrimaiteGame
from primaite.session.environment import PrimaiteGymEnv

# PrimAITE-Mail components
from primaite_mail.game.agents.green_mail_agent import GreenMailAgent
from primaite_mail.simulator.software.smtp_server import SMTPServer
from primaite_mail.simulator.software.email_client import EmailClient

# Store original config to restore later
original_dev_mode = PRIMAITE_CONFIG["developer_mode"]["enabled"]
original_sys_logs = PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"]
original_agent_logs = PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"]

# Enable developer mode and system logs for debugging
PRIMAITE_CONFIG["developer_mode"]["enabled"] = True
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = True
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = True

print("✅ All imports successful - ready for green mail agent demonstration")
print("📧 PrimAITE-Mail extension loaded with Green Mail Agent")
print("🔧 Developer mode enabled for enhanced debugging")
print(f"📋 System logs enabled: {PRIMAITE_CONFIG['developer_mode']['output_sys_logs']}")

## Load Mail Scenario

We'll load our simple mail scenario configuration that includes three green mail agents (Alice, Bob, and Charlie) in a basic office email environment.

In [ ]:
# Load the simple mail scenario configuration
config_path = Path("../config/simple_mail_scenario.yaml")

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

print(f"📄 Loaded configuration from: {config_path}")
print(f"🎮 Game settings: {config['game']['max_episode_length']} steps max")
print(f"👥 Number of agents: {len(config['agents'])}")
print(f"🖥️  Number of nodes: {len(config['simulation']['network']['nodes'])}")

# Display agent summary
agent_table = PrettyTable(["Agent", "Type", "Node", "Sender Email", "Frequency"])
agent_table.align = "l"
agent_table.title = "Green Mail Agents Configuration"

for agent in config['agents']:
    if agent['type'] == 'green-mail-agent':
        agent_table.add_row([
            agent['ref'],
            agent['type'],
            agent['agent_settings']['node_name'],
            agent['agent_settings']['sender_email'],
            f"{agent['agent_settings']['email_frequency']} ± {agent['agent_settings']['email_variance']}"
        ])

print(agent_table)

## Agent Configuration Review

Let's examine the configuration of our green mail agents to understand their behavior patterns.

In [ ]:
# Create the game from configuration
game = PrimaiteGame.from_config(config)

print(f"🎮 Game created successfully")
print(f"👥 Total agents: {len(game.agents)}")
print(f"🖥️  Network nodes: {len(game.simulation.network.nodes)}")

# Display detailed agent configuration
config_table = PrettyTable(["Agent", "Send Prob", "Retrieve Prob", "Frequency", "Variance", "Recipients"])
config_table.align = "l"
config_table.title = "Agent Configuration Details"

for agent_name, agent in game.agents.items():
    if isinstance(agent, GreenMailAgent):
        settings = agent.config.agent_settings
        recipients_str = ", ".join(settings.recipients[:2])  # Show first 2 recipients
        if len(settings.recipients) > 2:
            recipients_str += f" (+{len(settings.recipients)-2} more)"
        
        config_table.add_row([
            agent_name,
            f"{settings.send_probability:.1f}",
            f"{settings.retrieve_probability:.1f}",
            settings.email_frequency,
            settings.email_variance,
            recipients_str
        ])

print(config_table)

## Run Email Simulation

Now we'll run the simulation for several steps to observe the green mail agents in action. We'll monitor their email sending behavior and track the system state.

In [ ]:
# Initialize mailboxes for all users
mail_server = game.simulation.network.get_node_by_hostname("mail_server")
smtp_server = mail_server.software_manager.software.get("smtp-server")

if smtp_server:
    # Create mailboxes for all users
    users = ["alice", "bob", "charlie", "admin"]
    for user in users:
        response = game.simulation.apply_request([
            "network", "node", "mail_server", "service", "smtp-server", "create_mailbox",
            {"username": user}
        ])
        print(f"📫 Created mailbox for {user}: {response.status}")

print("\n🚀 Starting email simulation...")

In [ ]:
# Run simulation for 50 steps and track agent activity
steps_to_run = 50
agent_activity = {name: [] for name in game.agents.keys()}

print(f"⏱️  Running simulation for {steps_to_run} steps...\n")

# ✅ FIXED - Single call
for step in range(steps_to_run):
    game.step()  # Let game handle get_action() internally
    
    # Record activity from agent history
    for agent_name, agent in game.agents.items():
        if agent.history:
            latest_action = agent.history[-1]
            # Process the actual executed action

            if latest_action != "do-nothing":
                agent_activity[agent_name].append({
                    'step': step,
                    'action': latest_action,
                    'params': latest_action.parameters
                })
                print(f"📧 Step {step:2d}: {agent_name} sending email")

print(f"\n✅ Simulation completed after {steps_to_run} steps")

## System Logs and Debugging

Let's examine the system logs to understand what happened during the simulation and debug any issues.

In [ ]:
# Show system logs for debugging
print("🔍 SYSTEM LOGS FOR DEBUGGING:")
print("=" * 50)

# Show mail server logs
mail_server = game.simulation.network.get_node_by_hostname("mail_server")
if mail_server:
    print("\n📧 Mail Server System Logs:")
    mail_server.sys_log.show(last_n=15)
    
    # Show SMTP server logs specifically
    smtp_server = mail_server.software_manager.software.get("smtp-server")
    if smtp_server:
        print(f"\n📨 SMTP Server Status: {smtp_server.operating_state.name}")
        print(f"📨 SMTP Server Health: {smtp_server.health_state_actual.name}")

# Show client logs for debugging
for client_name in ["client_alice", "client_bob", "client_charlie"]:
    client_node = game.simulation.network.get_node_by_hostname(client_name)
    if client_node:
        print(f"\n💻 {client_name.title()} System Logs:")
        client_node.sys_log.show(last_n=8)
        
        # Show email client status
        email_client = client_node.software_manager.software.get("email-client")
        if email_client:
            print(f"   📧 Email Client Status: {email_client.operating_state.name}")
            print(f"   📧 Email Client Health: {email_client.health_state_actual.name}")
            print(f"   📧 Auto-start Config: {getattr(email_client.config, 'auto_start', 'Not set')}")

## Monitor Agent Activity

Now let's examine the activity patterns of our green mail agents and see how they performed during the simulation.

In [ ]:
# Display agent status and activity summary
activity_table = PrettyTable(["Agent", "Emails Sent", "First Email", "Last Email", "Activity Summary"])
activity_table.align = "l"
activity_table.title = "Agent Activity Summary"

for agent_name, agent in game.agents.items():
    if isinstance(agent, GreenMailAgent):
        activity = agent_activity[agent_name]
        first_email = activity[0]['step'] if activity else "None"
        last_email = activity[-1]['step'] if activity else "None"
        
        activity_table.add_row([
            agent_name,
            len(activity),
            first_email,
            last_email,
            f"{agent.emails_sent} sent, {agent.emails_retrieved} retrieved"
        ])

print(activity_table)

# Show detailed agent status
print("\n📊 Detailed Agent Status:")
for agent_name, agent in game.agents.items():
    if isinstance(agent, GreenMailAgent):
        print(f"\n--- {agent_name.upper()} ---")
        agent.show_status()

## Examine Email Traffic

Now let's look at the actual emails that were sent and received during our simulation by examining the mail server's mailboxes.

In [ ]:
# Display mail server status and mailbox contents
if smtp_server:
    print("📬 SMTP Server Status:")
    smtp_server.show()
    
    print("\n📫 Mailbox Contents:")
    smtp_server.show_mailbox()
    
    # Debug mailbox manager state
    print("\n🔍 Mailbox Manager Debug Info:")
    if smtp_server.mailbox_manager:
        print(f"   Total mailboxes: {len(smtp_server.mailbox_manager.mailboxes)}")
        for username, mailbox in smtp_server.mailbox_manager.mailboxes.items():
            messages = mailbox.get_messages()
            print(f"   {username}: {len(messages)} messages")
    else:
        print("   ❌ No mailbox manager found!")
else:
    print("❌ SMTP server not found")

In [ ]:
# Show individual mailbox contents for each user
users_to_check = ["alice", "bob", "charlie", "admin"]

for user in users_to_check:
    print(f"\n{'='*60}")
    print(f"📧 {user.upper()}'S MAILBOX")
    print(f"{'='*60}")
    
    if smtp_server:
        smtp_server.show_mailbox(user)
        
        # Show first message if any exist
        mailbox = smtp_server.mailbox_manager.get_mailbox(user)
        if mailbox:
            messages = mailbox.get_messages()
            if messages:
                print(f"\n📄 Sample Message (1 of {len(messages)}):")
                smtp_server.show_message(user, 1)
            else:
                print(f"📭 No messages in {user}'s mailbox")
        else:
            print(f"❌ Mailbox for {user} not found")

## Agent Behavior Analysis

Let's analyze the behavior patterns of our green mail agents and understand how they simulate realistic email usage.

In [ ]:
# Analyze timing patterns
import matplotlib.pyplot as plt
import numpy as np

# Create timeline visualization
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['blue', 'green', 'red']
y_positions = []
agent_names = []

for i, (agent_name, agent) in enumerate(game.agents.items()):
    if isinstance(agent, GreenMailAgent):
        activity = agent_activity[agent_name]
        if activity:
            steps = [item['step'] for item in activity]
            y_pos = [i] * len(steps)
            ax.scatter(steps, y_pos, c=colors[i % len(colors)], s=100, alpha=0.7, label=agent_name)
            y_positions.append(i)
            agent_names.append(agent_name)

ax.set_xlabel('Simulation Step')
ax.set_ylabel('Agent')
ax.set_title('Email Sending Timeline by Agent')
ax.set_yticks(y_positions)
ax.set_yticklabels(agent_names)
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate statistics
stats_table = PrettyTable(["Agent", "Avg Interval", "Min Interval", "Max Interval", "Total Emails"])
stats_table.align = "l"
stats_table.title = "Email Timing Statistics"

for agent_name, agent in game.agents.items():
    if isinstance(agent, GreenMailAgent):
        activity = agent_activity[agent_name]
        if len(activity) > 1:
            steps = [item['step'] for item in activity]
            intervals = [steps[i+1] - steps[i] for i in range(len(steps)-1)]
            avg_interval = np.mean(intervals)
            min_interval = min(intervals)
            max_interval = max(intervals)
        else:
            avg_interval = min_interval = max_interval = "N/A"
        
        stats_table.add_row([
            agent_name,
            f"{avg_interval:.1f}" if isinstance(avg_interval, float) else avg_interval,
            min_interval,
            max_interval,
            len(activity)
        ])

print(stats_table)

## Agent History Analysis

Let's examine the detailed history of agent actions to understand exactly what happened during the simulation.

In [ ]:
# Show agent action history
for agent_name, agent in game.agents.items():
    if isinstance(agent, GreenMailAgent) and agent.history:
        print(f"\n📋 {agent_name.upper()} ACTION HISTORY:")
        print(f"{'='*50}")
        
        # Use the built-in show_history method
        agent.show_history(ignored_actions=[])  # Show all actions including do-nothing
        
        # Show summary of successful vs failed actions
        successful_actions = sum(1 for item in agent.history if item.response.status == "success")
        total_actions = len(agent.history)
        
        print(f"\n📊 Success Rate: {successful_actions}/{total_actions} ({successful_actions/total_actions*100:.1f}%)")

## Summary and Extensions

This demonstration showed how the Green Mail Agent works in a PrimAITE simulation:

### Key Features Demonstrated

✅ **Realistic Email Patterns**: Agents send emails at configurable intervals with variance  
✅ **Multiple Recipients**: Each agent can send to multiple recipients from their contact list  
✅ **Varied Content**: Random selection of subjects and email templates  
✅ **Configurable Behavior**: Start times, frequencies, and limits are all configurable  
✅ **Integration**: Seamless integration with PrimAITE's email infrastructure  

### Potential Extensions

🔧 **Enhanced Realism**: Add time-of-day patterns, reply behaviors, or seasonal variations  
🔧 **Role-Based Behavior**: Different email patterns for managers, developers, support staff  
🔧 **Attachment Simulation**: Include file attachments in emails  
🔧 **Email Threads**: Implement reply chains and conversation threads  
🔧 **Security Scenarios**: Add phishing detection, spam filtering, or malware scenarios  

### Configuration Tips

💡 **Timing**: Use variance to create more realistic, non-mechanical patterns  
💡 **Recipients**: Include a mix of internal and external email addresses  
💡 **Content**: Customize email templates to match your scenario's context  
💡 **Timing**: Adjust email_frequency and email_variance for realistic patterns  
💡 **Monitoring**: Use the built-in `.show()` methods to observe system behavior  

The Green Mail Agent provides a solid foundation for creating realistic email traffic in cybersecurity simulations, supporting both normal operations and attack scenario development.

In [ ]:
# Returning to the original developer mode configuration.
PRIMAITE_CONFIG["developer_mode"]["enabled"] = original_dev_mode
PRIMAITE_CONFIG["developer_mode"]["output_sys_logs"] = original_sys_logs
PRIMAITE_CONFIG["developer_mode"]["output_agent_logs"] = original_agent_logs